In [8]:
import pandas as pd
import shutil
import os

In [9]:
%%bash
git clone https://github.com/Jaehun-Kim22/SCIC_Project.git

Cloning into 'SCIC_Project'...


In [10]:
## loads excel
xls = pd.ExcelFile('SCIC_Project/assets/data.xlsx')
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 448 kB 46.0 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [11]:
priority_dict = {}

for i in range(len(priority['의도명(유형)'])):
  priority_dict[priority['의도명(유형)'][i]] = priority['우선순위'][i]
print(priority_dict)

label_level3 = {}
level3_cnt = 0
label_level3['None'] = -1
for i in priority['의도명(유형)']:
  level = i.split('>')
  if len(level) == 3 and level[2] not in label_level3:
    label_level3[level[2]] = level3_cnt
    level3_cnt += 1
print(label_level3)


{'칭찬>고객서비스>상담원': 1, '칭찬>고객서비스>상담시스템': 2, '칭찬>고객서비스': 3, '불만>고객서비스>상담원': 4, '불만>고객서비스>상담시스템': 5, '불만>고객서비스': 6, '칭찬>삼성카드>혜택': 7, '칭찬>삼성카드>할부금융상품': 8, '칭찬>삼성카드>커뮤니티서비스': 9, '칭찬>삼성카드>카드이용/결제': 10, '칭찬>삼성카드>카드상품': 11, '칭찬>삼성카드>청구입금': 12, '칭찬>삼성카드>심사/한도': 13, '칭찬>삼성카드>생활편의서비스': 14, '칭찬>삼성카드>상담/채널': 15, '칭찬>삼성카드>리스렌탈상품': 16, '칭찬>삼성카드>라이프서비스': 17, '칭찬>삼성카드>금융상품': 18, '칭찬>삼성카드>고객정보관리': 19, '칭찬>삼성카드>가맹점매출/승인': 20, '칭찬>삼성카드>가맹점대금': 21, '칭찬>삼성카드>가맹점계약': 22, '칭찬>삼성카드': 23, '칭찬>기타': 24, '불만>삼성카드>혜택': 25, '불만>삼성카드>할부금융상품': 26, '불만>삼성카드>커뮤니티서비스': 27, '불만>삼성카드>카드이용/결제': 28, '불만>삼성카드>카드상품': 29, '불만>삼성카드>청구입금': 30, '불만>삼성카드>심사/한도': 31, '불만>삼성카드>생활편의서비스': 32, '불만>삼성카드>상담/채널': 33, '불만>삼성카드>리스렌탈상품': 34, '불만>삼성카드>라이프서비스': 35, '불만>삼성카드>금융상품': 36, '불만>삼성카드>고객정보관리': 37, '불만>삼성카드>가맹점매출/승인': 38, '불만>삼성카드>가맹점대금': 39, '불만>삼성카드>가맹점계약': 40, '불만>삼성카드': 41, '불만>기타': 42, '중립': 43, '폐기': 44}
{'None': -1, '상담원': 0, '상담시스템': 1, '혜택': 2, '할부금융상품': 3, '커뮤니티서비스': 4, '카드이용/결제': 5, '카드상품': 6, '청구입금': 7, '심사/한도': 8, '생활편의서비스': 

In [7]:
import requests
import json
import sys
import re
import xml.etree.ElementTree as ET
import copy

base_url = 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'

_agent = requests.Session()
PY3 = sys.version_info[0] == 3


def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>', '')
    if not PY3:
        text = text.encode('utf-8')

    result = ''.join(ET.fromstring(text).itertext())

    return result


def check(text):

    if isinstance(text, list):
        result = []
        for item in text:
            checked = check(item)
            result.append(checked)
        return result

    payload = {
        '_callback': 'window.__jindo2_callback._spellingCheck_0',
        'q': text
    }

    headers = {
        'user-agent': 'https://github.com/michael960725',
        'referer': 'https://naver.com/',
    }

    r = _agent.get(base_url, params=payload, headers=headers)
    r = r.text[42:-2]

    data = json.loads(r)
    html = data['message']['result']['html']
    return _remove_tags(html)


parsed_text_list = []
training_data_deepcopy = copy.deepcopy(training_data)
for i in training_data_deepcopy['발화']:
  new_text = re.sub('[^가-힣a-zA-Z0-9]', " ", i)
  parsed_text_list.append(check(new_text))

In [8]:

# training_data_copy_kkma = copy.deepcopy(training_data)
cnt = 0
column_label_level3 = []
for ground_truth in training_data['최종분류(우선순위 가장 높은것 선택)']:
  level = ground_truth.split('>')
  if len(level) == 3:
    column_label_level3.append(label_level3[level[2]])
  else:
    column_label_level3.append(label_level3['None'])
  cnt += 1
data = {'발화': parsed_text_list, 'label': column_label_level3}
label_for_level3 = pd.DataFrame(data)
# df = pd.DataFrame(np.random.randn(100, 2))
print(label_for_level3)
# training_data_copy_kkma['label'] = column_label_level3
# print(training_data_copy_kkma['최종분류(우선순위 가장 높은것 선택)'], training_data_copy_kkma['label'])


                                                    발화  label
0                                     상담직원과 빠른연결했슴좋겠어요      1
1                                안 기다리고  빨리 상담되었으면 좋겠다      1
2                                  너무 오래 기다리고 상담이 늦어져요      1
3                                  상담직원과  통화가  매우  힘들다      1
4                                 상담받으려면 오래 기다리는 게 불편함      1
..                                                 ...    ...
995  카드 분실된 지 1분 만에 카드를 통해서 연락받아 바로 찾았어요 잃어버린지도 몰랐는...      6
996         수시로 6개월 무이자 할부와 링크 혜택이 다른 카드와 구별되는 기능이 편리함      2
997                             무이자 할부 자주 많이 주셔서 감사합니다      2
998                        무이자 혜택 만족 긴 할부 만족 잘 쓰고 있습니다      2
999                                무이자 혜택이 많아 잘 쓰고 있어요      2

[1000 rows x 2 columns]


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(label_for_level3, test_size=0.2)

In [11]:
import numpy as np
train.to_csv(r'train.txt', header=None, index=None, sep='\t', mode='a')
test.to_csv(r'test.txt', header=None, index=None, sep='\t', mode='a')
# np.savetxt(r'c:Wtest.txt', test.values, fmt='%s')
# 이 파일들은 올려만 놓고 참조하는 방식

In [9]:
## Kkma sentence
from konlpy.tag import Kkma
import copy
import time
kkma = Kkma()


def perform_Kkma(text):
  temp_kkma = kkma.sentences(text)
  if [text] != temp_kkma:
    new_string_to_add = ''
    new_string_to_add += ' '.join(temp_kkma)
    return new_string_to_add
  else:
    return text


s = time.time()

training_data_copy_kkma = copy.deepcopy(training_data)

cnt = 0
column_added_kkma = []
for text in training_data['발화']:
  column_added_kkma.append(perform_Kkma(text))
  cnt += 1
training_data_copy_kkma['Kkma'] = column_added_kkma

print(training_data.values[:, 2][:20])
print(training_data_copy_kkma.values[:, 10][:20])
e = time.time()

print(e - s)

['상담직원과 빠른연결했슴좋겠어요' '안기다리고. 빨리상담되었음좋겠다' '너무 오래기다리고 상담이 늦어져요'
 '상담직원과  통화가  매우  힘들다' '상담받으려면오래기다리는게불편함' '상담하려면 많이기다려야하는게 불편합니다.'
 '상담원이통화중이라많이기다려야해요' '상담직원과의연결이 빠르게연결되면촣겠습니다' 'ars상담직원넘많이기다려야하는불편해요'
 '상담원 통화가너믜힘듬' '상담원과의 연결이빨리되었으면감사힘니다' '상담원이 영결이많이힘이듬'
 '5번질문에 대한 답변은 상담직원 연결되기까지 기다리는것말곤없는듯'
 '상담을 하려면 좀 많이기다려야 상담사분이 나오셔서 그게좀 불편하네요' 'Talk 상담이 다시 생겼으면..'
 '상담하려하는  기다리는시간    너무길다' '상담직원과 빠른속도로  연결이되길 원합니다' '상담직원과 빠른연결부탁드립니다'
 '상담직원과통화하기가힘들다' '상담한번받을려면 보통5분이상대기해야실질상담은2분미만인데.']
['상담 직원과 빠른연결했슴좋겠어요' '안 기다리고. 빨리 상담되었음 좋겠다' '너무 오래 기다리고 상담이 늦어져요'
 '상담 직원과 통화가 매우 힘들다' '상담 받으려면 오래 기다리는 게 불편함' '상담하려면 많이 기다려야 하는 게 불편합니다.'
 '상담원이 통화 중이라 많이 기다려야 해요' '상담 직원 과의 연결이 빠르게 연결되면 촣겠습니다'
 'ars 상담 직원 넘 많이 기다려야 하는 불편해요' '상담원 통화가 너 믜 힘 듬'
 '상담원과의 연결이 빨리 되었으면 감사 힘니다' '상담원이 영결이 많이 힘이 듬'
 '5번 질문에 대한 답변은 상담 직원 연결되기까지 기다리는 것 말곤 없는 듯'
 '상담을 하려면 좀 많이 기다려야 상담 사분이 나오셔서 그게 좀 불편하네요' 'Talk 상담이 다시 생겼으면..'
 '상담하려 하는 기다리는 시간 너무 길다' '상담 직원과 빠른 속도로 연결이 되길 원합니다' '상담 직원과 빠른 연결 부탁드립니다'
 '상담 직원과 통화하기가 힘들다' '상담한 번받 을려 면 보통 5분 이상 대기해야 실질

In [10]:
!pip install transformers
from transformers import BertModel, ElectraTokenizer
model = BertModel.from_pretrained('skt/kobert-base-v1')
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer.encode("한국어 모델을 공유합니다.")

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
     |████████████████████████████████| 636 kB 54.2 MB/s 
     |████████████████████████████████| 3.3 MB 41.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


[2, 11229, 6918, 4292, 7824, 17788, 18, 3]